In [27]:
import tensorflow as tf
import os
tf.random.set_seed(42)
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback  #
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Input

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Defining Path
DATASET_PATH = "/content/drive/MyDrive/dataset"

In [7]:
# Define image size and batch size
IMAGE_SIZE = (224,224) # Resize all images to 224x224 pixel values
BATCH_SIZE = 32        # Number of images per batch

In [8]:
# Load dataset from directory
dataset = image_dataset_from_directory(
    DATASET_PATH,
    labels='inferred', # Infer labels from folder names
    label_mode="int",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 9786 files belonging to 4 classes.


In [9]:
# Split the dataset into training, validation, and test sets
train_size = 0.7
val_size = 0.2
test_size = 0.1

In [10]:
# Getting the number of batches
total_batches = tf.data.experimental.cardinality(dataset).numpy()
train_batches = int(total_batches * train_size)
val_batches = int(total_batches * val_size)
print(total_batches,train_batches,val_batches)

306 214 61


In [11]:
# Split the data
train_dataset = dataset.take(train_batches)
remaining = dataset.skip(train_batches)
val_dataset = remaining.take(val_batches)
test_dataset = remaining.skip(val_batches)


In [12]:
for image, label in train_dataset.take(1):
    print(f"Image shape: {image.shape}, Label: {label.numpy()}")

Image shape: (32, 224, 224, 3), Label: [1 2 2 2 2 1 1 1 0 2 3 2 2 3 0 0 2 2 2 2 0 2 2 1 0 2 0 1 1 1 2 1]


In [13]:
print(f"Label shape: {label.shape}, Unique labels: {set(label.numpy())}")

Label shape: (32,), Unique labels: {0, 1, 2, 3}


In [14]:
# Normalize the pizel values to [0,1]
def normalize(image,label):
    image = tf.cast(image,tf.float32) / 255.0
    return image,label

In [15]:
# Prefetch for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [16]:
import tensorflow.keras.backend as K

# Custom function to restrict output between 0 and 4
def bounded_relu(x):
    return K.clip(x, 0, 4)


In [17]:
# Define the input shape
INPUT_SHAPE = (224,224,3) # Image size with 3 color channels (RGB)

In [25]:
# Number of training epochs
EPOCHS = 20 # Starting with 10 (Later change accordingly during hyperr parmeter tunning)
CHECKPOINT_PATH = "best_model.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "best_model.keras",  # Use .keras instead of .h5
    save_best_only=True,
    monitor="val_loss",
    mode="min",
    verbose=1
)

In [20]:
# Load EfficientNetB0 as feature extractor
base_model = EfficientNetB0(include_top=False,weights='imagenet',input_shape=(224,224,3))
base_model.trainable = False # Freezing the pretrained layers

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [21]:
# Input layer
inputs = Input(shape=(224,224,3))
x = base_model(inputs,training=False)
x = GlobalAveragePooling2D()(x)  # Convert feature maps into a single vector
x = Dropout(0.3)(x)  # Regularization
# Classification Head (Softmax for 5 classes)
classification_output = Dense(5,activation='softmax',name='classification_output')(x)

# Regression Head (Sigmoid * 4 to bound output between 0-4)
regression_output = Dense(1,activation='relu',name='regression_output')(x)  # 'sigmoid' activation can be used as the regression values are bounded
regression_output = tf.keras.layers.Lambda(lambda x:x*4,name='bounded_regression_output')(regression_output)

In [22]:
# Defined the final model
model = Model(inputs,outputs=[classification_output,regression_output])

In [32]:
# Compile with improved loss function
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss={'classification_output':'sparse_categorical_crossentropy','bounded_regression_output':tf.keras.losses.huber},
    metrics={'classification_output':'accuracy','bounded_regression_output':'mae'}
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb0            │ (None, 7, 7, 1280)     │      4,049,571 │ input_layer_1[0][0]    │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 1280)           │              0 │ efficientnetb0[0][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 1280)           │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ regression_output (Dense) │ (None, 1)              │          1,281 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ classification_output     │ (None, 5)              │          6,405 │ dropout[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bounded_regression_output │ (None, 1)              │              0 │ regression_output[0][… │
│ (Lambda)                  │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,057,257 (15.48 MB)

 Trainable params: 7,686 (30.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [28]:
class ConvertTensorToFloatCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for key, value in logs.items():
                if isinstance(value, tf.Tensor):
                    logs[key] = float(value.numpy())  # Convert to float
                elif isinstance(value, np.ndarray):
                    logs[key] = value.tolist()  # Convert array to list


In [34]:
def preprocess_data(image, label):
    # Assuming label contains both classification and regression targets
    classification_label = label[0]  # First part for classification
    regression_label = label[1]      # Second part for regression
    return image, (classification_label, regression_label)

train_dataset = train_dataset.map(preprocess_data)
val_dataset = val_dataset.map(preprocess_data)
import numpy as np

# Convert labels to numpy array of dtype int
train_labels = np.array(train_labels, dtype=np.int32)
val_labels = np.array(val_labels, dtype=np.int32)

NameError: name 'train_labels' is not defined

In [33]:
# Now, use it in model.fit()
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[ConvertTensorToFloatCallback(), checkpoint_callback]
)

Epoch 1/20


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(), output.shape=(None, 5)

In [1]:
#Install tensorflow version compatible with colab
# !pip install tensorflow==2.15.0
# %%
import tensorflow as tf
import os
tf.random.set_seed(42)
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback  #
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Input
# %%
from google.colab import drive
drive.mount('/content/drive')
# %%
# Defining Path
DATASET_PATH = "/content/drive/MyDrive/dataset"
# %%
# Define image size and batch size
IMAGE_SIZE = (224,224) # Resize all images to 224x224 pixel values
BATCH_SIZE = 32        # Number of images per batch
# %%
# Load dataset from directory
dataset = image_dataset_from_directory(
    DATASET_PATH,
    labels='inferred', # Infer labels from folder names
    label_mode="int",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
# %%
# Split the dataset into training, validation, and test sets
train_size = 0.7
val_size = 0.2
test_size = 0.1
# %%
# Getting the number of batches
total_batches = tf.data.experimental.cardinality(dataset).numpy()
train_batches = int(total_batches * train_size)
val_batches = int(total_batches * val_size)
print(total_batches,train_batches,val_batches)
# %%
# Split the data
train_dataset = dataset.take(train_batches)
remaining = dataset.skip(train_batches)
val_dataset = remaining.take(val_batches)
test_dataset = remaining.skip(val_batches)

# %%
for image, label in train_dataset.take(1):
    print(f"Image shape: {image.shape}, Label: {label.numpy()}")
# %%
print(f"Label shape: {label.shape}, Unique labels: {set(label.numpy())}")
# %%
# Normalize the pizel values to [0,1]
def normalize(image,label):
    image = tf.cast(image,tf.float32) / 255.0
    return image,label
# %%
# Prefetch for performance
# Here we call .map(normalize) on our dataset
train_dataset = train_dataset.map(normalize).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(normalize).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(normalize).prefetch(buffer_size=tf.data.AUTOTUNE)
# %%
import tensorflow.keras.backend as K

# Custom function to restrict output between 0 and 4
def bounded_relu(x):
    return K.clip(x, 0, 4)

# %%
# Define the input shape
INPUT_SHAPE = (224,224,3) # Image size with 3 color channels (RGB)
# %%
# Number of training epochs
EPOCHS = 20 # Starting with 10 (Later change accordingly during hyperr parmeter tunning)
CHECKPOINT_PATH = "best_model.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "best_model.keras",  # Use .keras instead of .h5
    save_best_only=True,
    monitor="val_loss",
    mode="min",
    verbose=1
)
# %%
# Load EfficientNetB0 as feature extractor
base_model = EfficientNetB0(include_top=False,weights='imagenet',input_shape=(224,224,3))
base_model.trainable = False # Freezing the pretrained layers
# %%
# Input layer
inputs = Input(shape=(224,224,3))
x = base_model(inputs,training=False)
x = GlobalAveragePooling2D()(x)  # Convert feature maps into a single vector
x = Dropout(0.3)(x)  # Regularization
# Classification Head (Softmax for 5 classes)
classification_output = Dense(5,activation='softmax',name='classification_output')(x)

# Regression Head (Sigmoid * 4 to bound output between 0-4)
regression_output = Dense(1,activation='relu',name='regression_output')(x)  # 'sigmoid' activation can be used as the regression values are bounded
regression_output = tf.keras.layers.Lambda(lambda x:x*4,name='bounded_regression_output')(regression_output)
# %%
# Defined the final model
model = Model(inputs,outputs=[classification_output,regression_output])
# %%
# Compile with improved loss function
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss={'classification_output':'sparse_categorical_crossentropy','bounded_regression_output':tf.keras.losses.huber},
    metrics={'classification_output':'accuracy','bounded_regression_output':'mae'}
)
model.summary()
# %%
class ConvertTensorToFloatCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for key, value in logs.items():
                if isinstance(value, tf.Tensor):
                    logs[key] = float(value.numpy())  # Convert to float
                elif isinstance(value, np.ndarray):
                    logs[key] = value.tolist()  # Convert array to list

# %%
# We are not preprocessing the data here, as the labels are already correctly formatted
# train_dataset = train_dataset.map(preprocess_data)
# val_dataset = val_dataset.map(preprocess_data)

# # We are not converting these labels, as the labels are already present in the dataset
# train_labels = np.array(train_labels, dtype=np.int32)
# val_labels = np.array(val_labels, dtype=np.int32)
# %%
# Now, use it in model.fit()
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[ConvertTensorToFloatCallback(), checkpoint_callback]
)
# %%

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 9786 files belonging to 4 classes.
306 214 61
Image shape: (32, 224, 224, 3), Label: [1 2 2 2 2 0 2 2 0 2 2 2 2 2 2 2 2 2 2 3 1 2 2 2 1 2 0 2 0 0 2 2]
Label shape: (32,), Unique labels: {0, 1, 2, 3}
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 efficientnetb0 (Functional  (None, 7, 7, 1280)           4049571   ['input_2[0][0]']             
 )                                                                                                
                                                         

NameError: name 'np' is not defined